In [1]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

from backtesting.test import SMA, GOOG


class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()


bt = Backtest(GOOG, SmaCross, commission=0.002, exclusive_orders=True)
stats = bt.run()
bt.plot()

/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/test/__init__.py:8: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(join(dirname(__file__), filename),
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'


GridPlot(id='p1325', ...)

In [2]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()
from numpy import cos, linspace

x = linspace(-6, 6, 100)
y = cos(x)
p = figure(width=500, height=500)
p.circle(x, y, size=7, color="firebrick", alpha=0.5)
show(p)

Loading BokehJS ...

In [7]:
from backtesting.test import GOOG

GOOG.tail()

,Open,High,Low,Close,Volume
2013-02-25,802.3,808.41,790.49,790.77,2303900
2013-02-26,795.0,795.95,784.40,790.13,2202500
2013-02-27,794.8,804.75,791.11,799.78,2026100
2013-02-28,801.1,806.99,801.03,801.20,2265800
2013-03-01,797.8,807.14,796.15,806.19,2175400


In [2]:
from backtesting import Strategy, Backtest
from backtesting.lib import crossover
import pandas as pd
from backtesting.test import GOOG


def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()


class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()


bt = Backtest(GOOG, SmaCross, cash=1000, commission=0.02)
stats = bt.run()
bt.plot()
stats

/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                   97.067039
Equity Final [$]                     1174.218
Equity Peak [$]                      1812.965
Return [%]                            17.4218
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                    1.902025
Volatility (Ann.) [%]               26.714624
Sharpe Ratio                         0.071198
Sortino Ratio                        0.108095
Calmar Ratio                         0.040828
Max. Drawdown [%]                   -46.58615
Avg. Drawdown [%]                   -8.801849
Max. Drawdown Duration     1558 days 00:00:00
Avg. Drawdown Duration      121 days 00:00:00
# Trades                                   94
Win Rate [%]                        38.297872
Best Trade [%]                      54.346616
Worst Trade [%]                     -18.77208
Avg. Trade [%]                    

In [12]:
stats = bt.optimize(
    n1=range(5, 30, 5),
    n2=range(10, 70, 5),
    maximize="Equity Final [$]",
    constraint=lambda param: param.n1 < param.n2,
)

stats

Start                     2004-08-19 00:00:00
End                       2013-03-01 00:00:00
Duration                   3116 days 00:00:00
Exposure Time [%]                    96.32216
Equity Final [$]                    2559.5922
Equity Peak [$]                     4144.9768
Return [%]                          155.95922
Buy & Hold Return [%]              703.458242
Return (Ann.) [%]                   11.657001
Volatility (Ann.) [%]                31.03124
Sharpe Ratio                         0.375654
Sortino Ratio                        0.652434
Calmar Ratio                         0.219813
Max. Drawdown [%]                  -53.031433
Avg. Drawdown [%]                   -7.343924
Max. Drawdown Duration      969 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                   60
Win Rate [%]                        41.666667
Best Trade [%]                      49.072006
Worst Trade [%]                    -20.815177
Avg. Trade [%]                    

In [17]:
bt.plot(plot_volume=False, plot_pl=False)
stats["_strategy"]

/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/opt/conda/envs/default/lib/python3.10/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


<Strategy SmaCross(n1=10,n2=35)>